>### **This notebook is an evolitionary review of building nn models with keras. I just focussed on base modeling.**
>### **Final model can improve by EDA, Missing Value imputation, Feat eng, transformation and model tuning.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from scipy import stats
from scipy.stats import norm, skew

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, GroupKFold, GridSearchCV, StratifiedKFold

from sklearn.metrics import *
from sklearn import metrics


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense


import sys, os
import random 

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
    
from IPython import display, utils

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('max_colwidth', 400)


def set_seed(seed=4242):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

In [ ]:
def make_confusion_matrix(cf,
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=True,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    


    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names)==cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = ["{0:.2%}".format(value) for value in cf.flatten()/np.sum(cf)]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels,group_counts,group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0],cf.shape[1])


    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    if sum_stats:
        #Accuracy is sum of diagonal divided by total observations
        accuracy  = np.trace(cf) / float(np.sum(cf))

        #if it is a binary confusion matrix, show some more stats
        if len(cf)==2:
            #Metrics for Binary Confusion Matrices
            precision = cf[1,1] / sum(cf[:,1])
            recall    = cf[1,1] / sum(cf[1,:])
            f1_score  = 2*precision*recall / (precision + recall)
            stats_text = "\n\nAccuracy={:0.3f}\nPrecision={:0.3f}\nRecall={:0.3f}\nF1 Score={:0.3f}".format(
                accuracy,precision,recall,f1_score)
        else:
            stats_text = "\n\nAccuracy={:0.3f}".format(accuracy)
    else:
        stats_text = ""


    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize==None:
        #Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks==False:
        #Do not show categories if xyticks is False
        categories=False


    # MAKE THE HEATMAP VISUALIZATION
    plt.figure(figsize=figsize)
    sns.heatmap(cf,annot=box_labels,fmt="",cmap=cmap,cbar=cbar,xticklabels=categories,yticklabels=categories)

    if xyplotlabels:
        plt.ylabel('True label')
        plt.xlabel('Predicted label' + stats_text)
    else:
        plt.xlabel(stats_text)
    
    if title:
        plt.title(title)

In [ ]:
def read_data():
    train = pd.read_csv('../input/flu-data/H1N1_Flu_Vaccines.csv')
    #display(train.head())
    
    del train['respondent_id']
   
    """cats = train.describe(include=['O']).columns
    for c in cats:
        le=LabelEncoder()
        le.fit(list(train[c].astype(str)) + list(test[c].astype(str)))
        train[c] = le.transform(train[c].astype(str))
        test[c] = le.transform(test[c].astype(str))"""
    return train

In [ ]:
train = read_data()

In [ ]:
sns.countplot(train.h1n1_vaccine)

In [ ]:
sns.countplot(train.seasonal_vaccine)

### Flu Vaccine Binary

In [ ]:
train.describe(include=['O'])

In [ ]:
missing = train.isnull().sum()
missing = missing[missing>0]

miss = pd.DataFrame(missing, columns=['missing'])

miss = miss.reset_index()

print(miss.columns)

miss.sort_values(by='missing', ascending=False, inplace=True)

plt.figure(figsize=(15, 10))
sns.barplot(y = miss['index'], x= miss.missing, palette='bone')

In [ ]:
cats = [c for c in train.columns if train[c].dtypes=='object']

nums = train.select_dtypes(exclude='object').columns

nums

In [ ]:
for c in cats:
    le=LabelEncoder()
    train[c] = le.fit_transform(train[c].astype(str)) 
    

In [ ]:
train.head().T

In [ ]:
import matplotlib.style as style
style.use('seaborn-poster')
sns.set_style('ticks')
plt.subplots(figsize = (27,20))
## Plotting heatmap. 

# Generate a mask for the upper triangle (taken from seaborn example gallery)
mask = np.zeros_like(train.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


sns.heatmap(train.corr(), cmap=sns.diverging_palette(20, 220, n=200), annot=True, mask=mask, center = 0, );
## Give title. 
plt.title("Heatmap of all the Features", fontsize = 25);

In [ ]:
del train['h1n1_vaccine']
target = train.pop('seasonal_vaccine')

train.shape



In [ ]:
train.fillna(-999, inplace=True)

In [ ]:
model = Sequential()
model.add(Dense(12, input_shape=(train.shape[1],),   activation= 'relu' ))
model.add(Dense(8,  activation= 'relu' ))
model.add(Dense(1,  activation= 'sigmoid' ))
model.summary()

In [ ]:
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])

In [ ]:
model.fit(train, target, validation_split=0.33 , epochs=15, batch_size=10)

In [ ]:
scores = model.evaluate(train, target)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

### KFold Cross Validation

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cvscores = []
for train_ind, test_ind in kfold.split(train, target):
# create model
    model = Sequential()
    model.add(Dense(12, input_shape=(train.shape[1],),   activation= 'relu' ))
    model.add(Dense(8,  activation= 'relu' ))
    model.add(Dense(1,  activation= 'sigmoid' ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    # Fit the model
    model.fit(train.iloc[train_ind], target.iloc[train_ind], epochs=15, batch_size=10, verbose=1)
    # evaluate the model
    scores = model.evaluate(train.iloc[test_ind], target.iloc[test_ind], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)


In [ ]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

### AUC

In [ ]:
import gc
import tensorflow as tf
from keras.layers import Dense, Input
from collections import Counter
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers import Dropout

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=45)
cvscores = []
auc_score = []
oof_nn= np.zeros((len(train)))
for fold_, (train_ind, test_ind) in enumerate(folds.split(train, target)):
# create model
    print('fold : --------------', fold_)
    
    trn_x, trn_y = train.iloc[train_ind], target.iloc[train_ind]
    val_x, val_y = train.iloc[test_ind], target.iloc[test_ind]

    model = Sequential()
    model.add(Dense(12, input_shape=(train.shape[1],),   activation= 'relu' ))
    model.add(Dense(8,  activation= 'relu' ))
    model.add(Dense(1,  activation= 'sigmoid' ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    # Fit the model
    model.fit(train.iloc[train_ind], target.iloc[train_ind], epochs=5, batch_size=10, verbose=1)
    val_preds = model.predict(val_x)
    
    print("AUC = {}".format(metrics.roc_auc_score(val_y,val_preds)))
    auc_score.append(metrics.roc_auc_score(val_y,val_preds))
    scores = model.evaluate(train.iloc[test_ind], target.iloc[test_ind], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    oof_nn[test_ind] =  val_preds.ravel()
    print(oof_nn[0:10])
    
    cvscores.append(scores[1] * 100)
K.clear_session()
gc.collect()
print("Accuracy Mean: ------->" , (np.mean(cvscores)))
print('AUC mean: ------->', np.mean(auc_score))

### Scale data 

In [ ]:
ss= StandardScaler()
train = ss.fit_transform(train)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics 
import gc
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=45)
cvscores = []
auc_score = []
oof_nn= np.zeros((len(train)))
for fold_, (train_ind, test_ind) in enumerate(folds.split(train, target)):
# create model
    print('fold : --------------', fold_)
    
    trn_x, trn_y = train[train_ind], target.iloc[train_ind]
    val_x, val_y = train[test_ind], target.iloc[test_ind]

    model = Sequential()
    model.add(Dense(12, input_shape=(train.shape[1],),   activation= 'relu' ))
    model.add(Dense(8,  activation= 'relu' ))
    model.add(Dense(1,  activation= 'sigmoid' ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
    # Fit the model
    model.fit(train[train_ind], target.iloc[train_ind], epochs=5, batch_size=10, verbose=1)
    val_preds = model.predict(val_x)
    
    print("AUC = {}".format(metrics.roc_auc_score(val_y,val_preds)))
    auc_score.append(metrics.roc_auc_score(val_y,val_preds))
    scores = model.evaluate(train[test_ind], target.iloc[test_ind], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    oof_nn[test_ind] =  val_preds.ravel()
    print(oof_nn[0:10])
    
    cvscores.append(scores[1] * 100)
K.clear_session()
gc.collect()
print("Accuracy Mean: ------->" , (np.mean(cvscores)))
print('AUC mean: ------->', np.mean(auc_score))

### AUC through epoches

In [ ]:
def fallback_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except:
        return 0.5


def auc(y_true, y_pred):
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics 
import gc
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=45)
cvscores = []
auc_score = []
oof_nn= np.zeros((len(train)))
for fold_, (train_ind, test_ind) in enumerate(folds.split(train, target)):
# create model
    print('fold : --------------', fold_)
    
    trn_x, trn_y = train[train_ind], target.iloc[train_ind]
    val_x, val_y = train[test_ind], target.iloc[test_ind]

    model = Sequential()
    model.add(Dense(12, input_shape=(train.shape[1],),   activation= 'relu' ))
    model.add(Dense(8,  activation= 'relu' ))
    model.add(Dense(1,  activation= 'sigmoid' ))
    # Compile model
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[auc])
    # Fit the model
    model.fit(train[train_ind], target.iloc[train_ind], epochs=5, batch_size=10, verbose=1)
    val_preds = model.predict(val_x)
    
    print("AUC = {}".format(metrics.roc_auc_score(val_y,val_preds)))
    auc_score.append(metrics.roc_auc_score(val_y,val_preds))
    scores = model.evaluate(train[test_ind], target.iloc[test_ind], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    oof_nn[test_ind] =  val_preds.ravel()
    print(oof_nn[0:10])
    
    cvscores.append(scores[1] * 100)
K.clear_session()
gc.collect()
print("Accuracy Mean: ------->" , (np.mean(cvscores)))
print('AUC mean: ------->', np.mean(auc_score))

### Better Network Topology

In [ ]:
import gc
import tensorflow as tf
from keras.layers import Dense, Input
from collections import Counter
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers import Dropout

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=142)
cvscores = []
auc_score = []
oof_nn= np.zeros((len(train)))

for fold_, (train_ind, test_ind) in enumerate(folds.split(train, target)):
# create model
    print('fold : |||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||>', fold_)
    
    trn_x, trn_y = train[train_ind], target.iloc[train_ind]
    val_x, val_y = train[test_ind], target.iloc[test_ind]

    model = Sequential()
    model.add(Dense(64, input_shape=(train.shape[1],),   activation= 'relu' ))
    model.add(Dropout(0.2))
    model.add(Dense(16,  activation= 'relu' ))
    model.add(Dropout(0.2))
    model.add(Dense(1,  activation= 'sigmoid' ))
    # Compile model
    #opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(loss= 'binary_crossentropy' , optimizer= 'Adam' , metrics=[auc])
    # Fit the model
    cp = callbacks.ModelCheckpoint(filepath="cp.hdf5", monitor="val_auc",  verbose=0,
        save_best_only=False, save_weights_only=False,  mode="auto")
    
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=20,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=True)

    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                      patience=3, min_lr=1e-6, mode='max', verbose=1)
    
    model.fit(trn_x, trn_y, validation_data=(val_x, val_y),callbacks= [cp, es, rlr],  epochs=100, batch_size=16, verbose=0)
    val_preds = model.predict(val_x)
    
    
    print("AUC = {}".format(metrics.roc_auc_score(val_y,val_preds)))
    auc_score.append(metrics.roc_auc_score(val_y,val_preds))
    #scores = model.evaluate(val_x, val_y, verbose=1)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    oof_nn[test_ind] =  val_preds.ravel()
    print(oof_nn[0:10])
    
    model.save_weights("model.h5")
    
    
K.clear_session()
gc.collect()

print('AUC mean: ------->', np.mean(auc_score))

In [ ]:
oof_nn

In [ ]:
oof_nn_rd = np.where(oof_nn > 0.5, 1, 0)
cf_matrix = confusion_matrix(target, oof_nn_rd) 

labels = ['True Neg','False Pos','False Neg','True Pos']
categories = ['Zero', 'One']
plt.style.use('seaborn-poster')
sns.set(font_scale=1.4)
make_confusion_matrix(cf_matrix, 
                      group_names=labels,
                      categories=categories, 
                      cmap='vlag', figsize=(12, 8))